In [6]:
import os
import csv
import requests
from datetime import datetime, timedelta
from calendar import monthrange

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm

In [5]:
from constants import FEDERAL_CITIES

In [14]:
df_stations = pd.read_csv('../../data/raw/federal/metadata/StationsNAPS-StationsSNPA.csv')
df_stations.columns

Index(['NAPS_ID', 'Station_Name', 'Status', 'Location_Address', 'City', 'P/T',
       'Postal_Code', 'Timezone', 'Latitude', 'Longitude', 'Elevation',
       'Start_Year', 'End_Year', 'Combined_Stations', 'Inlet_Height',
       'Network', 'SO2', 'CO', 'NO2', 'NO', 'NOX', 'O3', 'PM_25_Continuous',
       'PM_10_Continuous', 'PM_2.5_RM', 'PM10-2.5', 'PM2.5_Speciation', 'VOC',
       'Carbonyl', 'PAH', 'Site_Type', 'Urbanization', 'Neighbourhood',
       'Land_Use', 'Scale', 'PC', 'CD', 'CSD', 'CMA/CA', 'AQMS_Airzone',
       'Core_Site'],
      dtype='object')

Remove stations which have old data only (before 2000) or aren't in our list